# Stroke Prediction

The goal of this project is to determine which features of an individual’s health data are highly predictive of whether the individual would get a stroke or not.

In [ ]:
# import custom class and function
from project import DataPreparation, make_histogram

# Data Preparation (Part 1)

We'll be working with the Stroke Prediction dataset from Kaggle, which contains information on patients who had or didn't have a stroke.

## Import Dataset

First, we imported the dataset. We used a `try-except` block to check if it's a csv file.

In [ ]:
import pandas as pd

In [ ]:
try:
    df = pd.read_csv("healthcare-dataset-stroke-data.csv") # read csv file, save it as a dataframe
except csv.Error as error:
    print("Not a csv file") # if not a csv file, print error message

We created an instance `healthcare` of the class `DataPreparation` to look at the data.

In [ ]:
# create an instance of the custom class
healthcare = DataPreparation(df)
healthcare.df.head()

## Clean Data

To prep our data for exploratory analysis, we called `__clean_df__()` from `DataPreparation` to convert all the qualitative features to binary values.

We converted "work_type" to `is_employed`, "Residence_type" to `is_Rural`, and "smoking_status" to `has_smoked`. We dropped `id`, "Other" from `gender`, and all the NaN values.

In [ ]:
# clean the data by calling the 1st class function
healthcare.clean_df()
healthcare.df.head()

# Data Visualization

Before determining which features are the most relevant to predicting stroke, we made some hypotheses based on the simple graphic comparison between features and their according stroke cases.

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

## Histograms

The histograms show the quantitative features (`age`, `avg_glucose_level`, `bmi`) and the number of stroke and non-stroke cases.

In [ ]:
# call make_histogram() function
f = ["age", "avg_glucose_level", "bmi"]
make_histogram(healthcare.df, f)

1. The histograms of age and stroke and non-stroke cases show that there is a significant trend of people with higher age getting more stroke.
2. The second histograms of average glucose level and its stroke and non-stroke cases show that there is a mild trend of people who had stroke having higher average glucose level.
3. The third histograms of bmi and its stroke and non-stroke cases didn't show much significance between the feature and number of the cases.

### Exception Handling

If there are more or less than 3 features, raise `ValueError`, since the histograms are supposed to represent 3 specific features.

In [ ]:
# f = ["age", "bmi"]
# make_histogram(healthcare.df, f)

If the features aren't quantitative, raise `TypeError`. The function will try to detect values over 2, and if there is no value over 2, it will print an error message explaining that the list contains qualitative features.

In [ ]:
# f = ["age", "bmi", "gender"]
# make_histogram(healthcare.df, f)

Note: uncomment to test exception handling.

## Bargraphs

The bar graphs show the qualitative features (`gender`, `hypertension`, `heart_disease`, `ever_married`, `is_employed`, `is_Rural`, `has_smoked`). The left column shows stroke cases while the right column shows non-stroke cases.

In [ ]:
# filter the data by stroke vs non-stroke cases
stroke_true = healthcare.df["stroke"] == 1
stroke_false = healthcare.df["stroke"] == 0

In [ ]:
# create bar graphs
fig, axes = plt.subplots(7, 2, figsize = (10, 30))

sns.countplot(ax=axes[0,0], data=healthcare.df[stroke_true], x='gender').set(title='stroke') # left column
sns.countplot(ax=axes[0,1], data=healthcare.df[stroke_false], x='gender').set(title='no stroke') # right column
sns.countplot(ax=axes[1,0], data=healthcare.df[stroke_true], x='hypertension')
sns.countplot(ax=axes[1,1], data=healthcare.df[stroke_false], x='hypertension')
sns.countplot(ax=axes[2,0], data=healthcare.df[stroke_true], x='heart_disease')
sns.countplot(ax=axes[2,1], data=healthcare.df[stroke_false], x='heart_disease')
sns.countplot(ax=axes[3,0], data=healthcare.df[stroke_true], x='ever_married')
sns.countplot(ax=axes[3,1], data=healthcare.df[stroke_false], x='ever_married')
sns.countplot(ax=axes[4,0], data=healthcare.df[stroke_true], x='is_employed')
sns.countplot(ax=axes[4,1], data=healthcare.df[stroke_false], x='is_employed')
sns.countplot(ax=axes[5,0], data=healthcare.df[stroke_true], x='is_Rural')
sns.countplot(ax=axes[5,1], data=healthcare.df[stroke_false], x='is_Rural')
sns.countplot(ax=axes[6,0], data=healthcare.df[stroke_true], x='has_smoked')
sns.countplot(ax=axes[6,1], data=healthcare.df[stroke_false], x='has_smoked')

plt.show()

Based on the results, we hypothesize that the features most relevant to predicting stroke are `is_employed`, `hypertension`, and `has_smoked`. 

We were surprised to learn that for patients who had a stroke, **all** of them were employed. It's important to note, though, that most of the patients from the dataset are employed. Additionally, `has_smoked` doesn't seem to matter as much as we thought. But we can conclude that there are more patients who smoke compared to those who don't for stroke cases, and vice versa for non-stroke cases.

# Data Preparation (Part 2)

To prep our data for machine learning, we called `__clean_df_2__()` from `DataPreparation` to convert all the quantitative variables to binary values. 

We converted "age" to `is_Old`, "avg_glucose_level" to `has_high_glucose`, and "bmi" to `is_Overweight`.

In [ ]:
# call 2nd class function
healthcare.clean_df_2()
healthcare.df.head()

# Comparing Models

We wanted to train two models and compare which of the two is better for predicting stroke.

To prep for that, we called `__train_test_split__()` from `DataPreparation` to split our data into train and test sets. That way, we can see how well each model performs on unseen data.

In [ ]:
# call 3rd class function
X, y, X_train, X_test, y_train, y_test = healthcare.train_test_split()

## Model 1

We fitted a Logistic Regression model on the training data and tested the model's performance on the testing data.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# train logistic regression model
m = LogisticRegression(solver="liblinear")
m.fit(X_train, y_train)

Next, we computed the classification error for the training and test data to gauge the performance of the logistic regression model.

In [ ]:
train_score = m.score(X_train, y_train)
test_score = m.score(X_test, y_test)
print("train score: ", train_score)
print("test score: ", test_score)

The train and test score are both high and close to 1, so our model is pretty good. However, these scores may not be accurate as they are dependent on the train-test split (random process). To account for the randomness, let's do a 3-fold cross-validation.

In [ ]:
# calculate 3-fold cross validation score
cv_score = cross_val_score(m, X_train, y_train, cv=3).mean()
print(cv_score)

The 3-fold cross validation score is pretty high (nearly the same as the test score for one test-train split calculated previously) so we can conclude logistic regression is a good model.

## Model 2

But is there a better model? Let's train a Decision Tree Classifier model and compute cross-validation scores for max depth of 1-30. This will help us determine the depth that will give us the best Decision Tree Classifier model.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import tree

In [ ]:
def fit_tree(X, y, d):
    '''
    Args:
        X: training data
        y: testing data
        d: depth
    Returns:
        T: instance of decisiontree classifier
    '''
    T = tree.DecisionTreeClassifier(max_depth=d) # create an instance of decisiontree classifier
    T.fit(X, y) # train the decisiontree classifier model
    return T

In [ ]:
# train data with decisiontree classifier
fig, ax = plt.subplots(1, figsize= (10, 8))
depths = range(1, 30)
cv_scores = []

T = tree.DecisionTreeClassifier(max_depth = 3)

for d in depths:
    T = fit_tree(X_train, y_train, d)
    cv_score = cross_val_score(T, X_train, y_train, cv=3).mean() # compute 3-fold cv score
    
    cv_scores.append(cv_score)

ax.scatter(depths, cv_scores, label='cv score')
ax.set(xlabel="depth", ylabel= 'score')
plt.legend(fontsize=20)
plt.show()

min_arg = np.argmax(cv_scores)

print("best depth:", depths[min_arg])
print("highest CV score:", cv_scores[min_arg])

Logistic Regression and Decision Tree Classifier (of any depth 1-5) are both good models for our dataset. The cross validation scores for Decision Tree Classifier were nearly the same for all depths (from 1-30). However, models with depths 1-5 perform slightly better than those with higher depths. Although Logistic Regression and Decision Tree Classifier are equally good models (both have a cross validation score of about 0.95), we will use the Logistic Regression model to examine which features are most correlated with getting a stroke.

# Logistic Regression

Let's determine which features are the most relevant for predicting stroke using Logistic Regression.

## Correlation Coefficients

We stored the names of all the features in a list called `column_names`, which we'll iterate through for our graph. We also calculated the correlation coefficients between each feature and `stroke`.

In [ ]:
# calculate correlation coefficients
m.coef_

In [ ]:
# create a list of all the features
column_names = list(X.columns.values)
column_names

We created a graph that displays all the features and their corresponding correlation coefficients.

In [ ]:
plt.figure(figsize=(9,7))
plt.barh(column_names, m.coef_[0][:], 0.3) # plots features and coefficients
plt.title("Logistic Regression Model")
plt.xlabel("Coefficients")
plt.axvline(x=0, color='gray') # creates line at 0 for x-axis
plt.subplots_adjust(left=0.3)

plt.show()

The logistic regression coefficient **β** that is associated with each feature changes in log odds. This means that increasing the feature by 1 unit multiplies the chances of having the outcome by **eβ**. (https://quantifyinghealth.com/interpret-logistic-regression-coefficients/)

To understand this better, we calculated the odds ratio for all the coefficients.

## Odds Ratio

Since we're working with medical data, let's interpret the coefficients as odds ratios to associate features to stroke risk. The odds ratio is the ratio of the probabilities of two mutually exclusive outcomes. For example, the odds of smoking group getting a stroke vs the odds of non-smoking group getting a stroke.

In [ ]:
import math
import numpy as np

In [ ]:
# create an empty list
odds_ratio_L = []

# compute odds ratio for each feature
for i in m.coef_[0]:
    odds_ratio = math.e**i
    odds_ratio_L.append(odds_ratio) # add to list

# save as numpy array
odds_ratio_L = np.array(odds_ratio_L)
print("odds ratio: ", odds_ratio_L)